In [6]:
# Imports

import cv2
from PIL import Image

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import math as maths

import os
import random

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.nn import functional
#from torchsummary import summary
#import torchvision.transforms as transforms

from tqdm import tqdm
from IPython.display import clear_output

import wandb
from tqdm import tqdm
import pprint

from functions import Unwrap, label_oh_tf, import_imagedata, get_data, ImageProcessor
from loop_fns import loop, test_loop
from fns4wandb import build_optimizer, set_optimizer, set_lossfn, train, train_model, train_log, log_test_score
from architectures import build_net

In [2]:
wandb.login()

wandb: Currently logged in as: naughticalnonsence (antvis). Use `wandb login --relogin` to force relogin


True

In [ ]:
"""'distribution': 'log_uniform_values',
            'min': 1e-5,
            'max': 7e-5"""

In [30]:
"""# HP sweep

config = {
    'method': 'random',
    'metric':{
        'goal': 'minimize',
        'name': 'val_loss'},
    'parameters': {
        'dropout':{
            'values': [0.4] #0.5, 0.45, 0.55
        },
        'weight_decay':{
            'values': [4e-5,1e-5] #1e-5,2e-5, 3e-5,1e-4, 1e-6,2e-5, 3e-5
        },
        'epochs':{
            'value': 80
        },
        'lin_layer_size': {
            'values': [100]
        },
        'first_lin_lay':{
            'values':[10240] #1x10240 and 12288x100)
        },
        'optimizer': {
            'values': ['adam']
        },
        'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'log_uniform_values',
        'min': 0.000001,
        'max': 0.001
        },
        'kernal_size': {
            'values': [(3,5), (3,4), (4,5)] #,(3,5), (4,5), (4,6)  ### KERNAL SIZE AFFECTS CONV OUTPUT SIZE!
        },
        'loss_fn': {
            'values': ['MSE', 'CrossEntropy']
        },
    }
}

sweep_id = wandb.sweep(config, project='HPS_basedon_decent-sweep-24_UNwrapped_36113')


col_dict={
    'colour': 'colour',
    'size': [113,36]
}"""

Create sweep with ID: 36wsgole
Sweep URL: https://wandb.ai/antvis/HPS_basedon_decent-sweep-24_UNwrapped_36113/sweeps/36wsgole


In [3]:
# training

config = dict(
    epochs= 120, 
    learning_rate =1.7543891908426036e-05, #1e-5,
    dataset= 'IDSW',
    architecture ='CNN',
    optimizer= 'adam',
    weight_decay= 1.0e-05, #1e-4,
    loss_fn = 'MSE',
    dropout = 0.4,
    lin_layer_size = 100,
    first_lin_lay= 10240,
    kernal_size = [3,5],
    in_chan = 3
)




col_dict={
    'colour': 'colour',
    'size': [36,113]
}

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda:0 device


In [12]:
print(col_dict)

{'colour': 'colour', 'size': [36, 113]}


In [8]:
x_train, y_train, x_val, y_val, x_test, y_test = get_data()
                            # Common functions

"""def build_optimizer(network, optimizer, learning_rate, weight_decay=0):
    if optimizer == 'SGD':
        optimizer = torch.optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        if weight_decay == 0:
            optimizer = torch.optim.Adam(network.parameters(),
                               lr=learning_rate)
        optimizer = torch.optim.Adam(network.parameters(),
                               lr=learning_rate, weight_decay=weight_decay)
    return optimizer

def train_log(t_loss, v_loss, sample_count, epoch):
    wandb.log({'epoch': epoch,
              't_loss': t_loss,
              'v_loss': v_loss},
             step=sample_count)
    print(f'loss after {str(sample_count).zfill(5)} examples: {v_loss:.3f}')

                                # HP Sweep
def train(config=None):
    with wandb.init(config=config):
        config = wandb.config

        model = build_net(config.lin_layer_size,config.dropout, config.first_lin_lay, config.kernal_size).to(device)
        if config.loss_fn == 'MSE':
            loss_fn = nn.MSELoss()
        elif config.loss_fn == 'CrossEntropy':
            loss_fn = nn.CrossEntropyLoss()
        
        
        e_count = 0
        #optimizer = build_optimizer(network, config.optimizer, config.learning_rate, config.weight_decay)
        if e_count >= 20:
            optimizer = build_optimizer(model, config.optimizer, config.learning_rate, config.weight_decay)
        else:
            optimizer = build_optimizer(model, config.optimizer, config.learning_rate)
        
        for epoch in range(config.epochs):

            t_loss, predict_list, t_num_correct, model, optimizer = loop(model, x_train, y_train, epoch, loss_fn, device, col_dict, optimizer=optimizer)
            
            t_accuracy = (t_num_correct /len(x_train))*100
            
            v_loss, __, v_num_correct= loop(model, x_val, y_val, epoch, loss_fn, device,col_dict, train=False) 
            
            v_accuracy= (v_num_correct / len(x_val))*100
            
            t_avg_loss =t_loss/len(x_train)
            v_avg_loss = v_loss /len(x_val)
            
            e_count +=1
            
            wandb.log({'avg_train_loss': t_avg_loss, 'epoch':epoch})
            wandb.log({'avg_val_loss': v_avg_loss, 'epoch':epoch})
            wandb.log({'train_loss': t_loss, 'epoch':epoch})
            wandb.log({'val_loss': v_loss, 'epoch':epoch})
            wandb.log({'train_accuracy_%': t_accuracy})
            wandb.log({'val_accuracy_%': v_accuracy})
"""
                                #Training
            
"""def train_model(model, x_train, y_train, x_val, y_val,loss_fn, config):
    wandb.watch(model, loss_fn, log='all', log_freq=10)
    
    sample_count =0
    batch_count = 0
    e_count = 0
    
    for epoch in tqdm(range(config.epochs)):
        if e_count >= 20:
            optimizer = build_optimizer(model, config.optimizer, config.learning_rate, config.weight_decay)
        else:
            optimizer = build_optimizer(model, config.optimizer, config.learning_rate)
            
        #train
        t_loss, predict_list, t_num_correct, model, optimizer = loop(model, x_train, y_train, epoch, loss_fn, device, col_dict, optimizer=optimizer)
        sample_count += len(x_train)
       
        # validation
        v_loss, __, v_num_correct= loop(model, x_val, y_val, epoch, loss_fn, device,col_dict, train=False) 
        batch_count +=1
        
        if (batch_count +1)%25 ==0:
            train_log(t_loss,v_loss, sample_count, epoch)
        e_count +=1
        clear_output()"""
        
    
"""if col_dict['size'][0]!= col_dict['size'][1]:
    wrap = 'UNwrapped'
elif col_dict['size'][0]== col_dict['size'][1]:
    wrap = 'Wrapped'"""

def pipeline(hp, col_dict): 
    if col_dict['size'][0]== col_dict['size'][1]:
        wrap = 'wrapped'
    elif col_dict['size'][0]!= col_dict['size'][1]:
        wrap = 'unwrapped'
        
    title = f"{col_dict['colour']}_{wrap}_{col_dict['size']}"

    x_train, y_train, x_val, y_val, x_test, y_test = get_data()
    
    with wandb.init(project=title, config=hp):
        config = wandb.config
        model = build_net(lin_layer_size=config.lin_layer_size,
                          dropout=config.dropout,
                          first_lin_lay=config.first_lin_lay,
                          ks= config.kernal_size,
                          in_chan=config.in_chan).to(device)
        loss_fn = nn.MSELoss()
        
        train_model(model, x_train, y_train, x_val, y_val,loss_fn, config, col_dict)
        test_loop(model, x_test, y_test, loss_fn, device, col_dict,title)
        
    return model
        

In [15]:
print(col_dict)
if col_dict['size'][0]== col_dict['size'][1]:
    print('same')
elif col_dict['size'][0]!= col_dict['size'][1]:
    print('different')

{'colour': 'colour', 'size': [36, 113]}
different


In [9]:
model = pipeline(config, col_dict)

  0%|          | 0/120 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "<ipython-input-8-759d35540b03>", line 117, in pipeline
    train_model(model, x_train, y_train, x_val, y_val,loss_fn, config, col_dict)
  File "/its/home/nn268/optics/fns4wandb.py", line 120, in train_model
    t_loss, predict_list, t_num_correct, model, optimizer = loop(model, x_train, y_train, epoch, loss_fn, device, col_dict, optimizer=optimizer)
NameError: name 'loop' is not defined


Widget Javascript not detected.  It may not be installed or enabled properly.


NameError: name 'loop' is not defined

In [ ]:
#wandb.agent(sweep_id, train, count=25)

wandb: Agent Starting Run: ykh64eg1 with config:
wandb: 	dropout: 0.4
wandb: 	epochs: 80
wandb: 	first_lin_lay: 10240
wandb: 	kernal_size: [3, 5]
wandb: 	learning_rate: 0.0008285560907048908
wandb: 	lin_layer_size: 100
wandb: 	loss_fn: MSE
wandb: 	optimizer: adam
wandb: 	weight_decay: 4e-05
